In [1]:
import os
import random
import torchbnn as bnn
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search.bohb import TuneBOHB
import ray
from ray import tune
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import copy
import numpy as np
import pandas as pd
import os
import torch
import tqdm
from torch.utils.data import DataLoader
from torchmetrics.regression import R2Score
import ray.train as train
from ray.air import session
from ray.train.torch import TorchTrainer
from ray.air.config import ScalingConfig
import pyarrow

C:\Users\omondiagbep\Anaconda3\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\omondiagbep\Anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\omondiagbep\Anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\omondiagbep\Anaconda3\envs\pytorch\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
PREDICT_PROPERTIES=['ph_water']
def get_spectra_data(train_csv, test_csv=None,mineral=False,target_dataframe=None,over_write_csv=False):

    SPECTRA_COLUMN_STARTING = "nir."
    ID=['labSampleId']

    # check if this is a target and make the target csv to be the same with the global file used for training
    if isinstance(train_csv, list):
        df_source = pd.concat(map(pd.read_csv, train_csv), ignore_index=True)
        #remove spectra signal which is more than 1
        print("removing out of bound spectra from source")

        #subset only the mineral from the lucas data
        if mineral:
            reflectance_out_of_bound = df_source.loc[df_source[df_source[df_source.columns[pd.Series(df_source.columns).str.startswith(SPECTRA_COLUMN_STARTING)]] > 1].dropna(
            how='all', axis=0).index]
            df_source = df_source.loc[set(df_source.index) - set(reflectance_out_of_bound.index)]

            if 'mineral' in df_source.columns:
                df_source=df_source.loc[df_source['mineral'] == 'mineral']


    if target_dataframe is not None:
        df_target = target_dataframe

    if isinstance(test_csv, list) and target_dataframe is None:
        df_target = pd.concat(map(pd.read_csv, test_csv), ignore_index=True)

        # remove spectra signal which is more than 1
        print("removing out of bound spectra from target")

        if mineral:
            reflectance_out_of_bound = df_target.loc[df_target[df_target[df_target.columns[
                pd.Series(df_target.columns).str.startswith(SPECTRA_COLUMN_STARTING)]] > 1].dropna(
                how='all', axis=0).index]

            df_target = df_target.loc[set(df_target.index) - set(reflectance_out_of_bound.index)]

    #common_cols = list(set.intersection(set(df_target), set(df_source)))
    #print(common_cols)

    common_cols = df_source.columns.intersection(df_target.columns)

    # use this list to perform column selection
    df_target_ = df_target[common_cols]
    df_source_ = df_source[common_cols]


    # extract properties for source and target
    df_source_ = df_source_.filter(regex='^(nir.|ph_water|labSampleId)')
    df_target_ = df_target_.filter(regex='^(nir.|ph_water|labSampleId)')



    # Define the character to remove
    character_to_remove = 'nir.'

    # Remove the character from column names
    df_source_.columns = df_source_.columns.str.replace(character_to_remove, '')
    df_target_.columns = df_target_.columns.str.replace(character_to_remove, '')


    # Extract column names with numbers greater than the threshold
    source_columns_to_drop = [col for col in df_source_.filter(regex='^(nir.)').columns if int(col) > 4000]
    target_columns_to_drop = [col for col in df_target_.filter(regex='^(nir.)').columns if int(col) > 4000]

    # Drop the selected columns from the DataFrame
    df_source_ = df_source_.drop(columns=source_columns_to_drop)
    df_target_ = df_target_.drop(columns=target_columns_to_drop)


    #split train datasest into train and amin_val_loss
    from sklearn.model_selection import train_test_split
    df_source_train, df_source_val= train_test_split(df_source_,  test_size=0.2, random_state=42)



    df_source_train_= df_source_train.drop(columns=[ "ph_water"])
    df_source_val_= df_source_val.drop(columns=["ph_water"])
    df_target__= df_target_.drop(columns=[ "ph_water"])

    df_source_spectral_train = get_spectra(df_source_train_)

    df_source_spectral_val = get_spectra(df_source_val_)
    print(len(df_source_spectral_train))
    print(len(df_source_spectral_val))


    df_target_spectral = get_spectra(df_target__)
    print(len(df_target_spectral))

    #this is use to trim the incoming data when we upload the model to ur api
    if over_write_csv:
        pd.DataFrame(df_source_spectral_train).head(2).to_csv("../Data/ph_source_file.csv", index=False)

    df_source_spectral_train=df_source_spectral_train.to_numpy()
    df_source_spectral_val=df_source_spectral_val.to_numpy()
    df_target_spectral=df_target_spectral.to_numpy()

    print(df_source_spectral_train.shape)

    for i in range(len(PREDICT_PROPERTIES)):
        if PREDICT_PROPERTIES[i] not in df_source_train.columns:
            df_source_train[PREDICT_PROPERTIES[i]] =df_source_train[PREDICT_PROPERTIES[i]]

        if PREDICT_PROPERTIES[i] not in df_source_val.columns:
            df_source_val[PREDICT_PROPERTIES[i]] =df_source_val[PREDICT_PROPERTIES[i]]

        if PREDICT_PROPERTIES[i] not in df_target_.columns and PREDICT_PROPERTIES[i] in  df_target:
            df_target_[PREDICT_PROPERTIES[i]] =df_target[PREDICT_PROPERTIES[i]]

    # remove the 2500 column in the USA/NZ data
    #df_source = df_source.drop([SPECTRA_COLUMN_STARTING, SPECTRA_COLUMN_STARTING + '2500'], axis=1, errors='ignore')
    #df_target = df_target.drop([SPECTRA_COLUMN_STARTING, SPECTRA_COLUMN_STARTING + '2500'], axis=1, errors='ignore')

    return df_source_train, df_source_val,df_target_, df_source_spectral_train,df_source_spectral_val, df_target_spectral


def get_spectra(df):

    filter_col = [col for col in df.columns]

    data = df[filter_col]

    #no preprocessing done
    # Ignore first 200 * 0.5 = 100 nm and  pick every 50 * 0.5 = 25 nm
    data=data.iloc[:, 2::10]

    return data

In [3]:
import pandas as pd

#change to match your the path where you have the cal and cal dataset
GLOBAL_CSV=[ "C:/Projects/SmapProjects/SpectraData2023/mir_ph_cali.csv"]
#df_sources, df_targets, df_sources_spectral, df_targets_spectral=get_spectra_data(train_csv=GLOBAL_CSV,target_dataframe=df)

TARGET_CSV=[ "C:/Projects/SmapProjects/SpectraData2023/mir_ph_val.csv"]
df_source_train, df_source_val,df_target, df_source_spectral_train, df_source_spectral_val,df_target_spectral=get_spectra_data(train_csv=GLOBAL_CSV,test_csv=TARGET_CSV,over_write_csv=True)

X_train_raw = df_source_spectral_train#.reshape(df_source_spectral_train.shape[0], df_source_spectral_train.shape[1], 1)
#X_train=np.log(1 / X_train)
y_train = np.array(df_source_train[PREDICT_PROPERTIES].values)

X_val_raw = df_source_spectral_val#.reshape(df_source_spectral_val.shape[0], df_source_spectral_val.shape[1], 1)
y_val= np.array(df_source_val[PREDICT_PROPERTIES].values)

X_test_raw =df_target_spectral#.reshape(df_target_spectral.shape[0], df_target_spectral.shape[1], 1)
y_test =df_target[PREDICT_PROPERTIES].values

removing out of bound spectra from source
removing out of bound spectra from target
2166
542
902
(2166, 340)


In [4]:
scaler = StandardScaler()
scaler.fit(X_train_raw)
scaler.fit(X_val_raw)
X_val = scaler.transform(X_val_raw)
X_train = scaler.transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [5]:
# Convert to 2D PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

In [6]:
#TEST Model
def evaluate(model,test_x,test,type="mae"):
    df =evaluate_soil_property(model,test_x,sample_size = 100)

    result_total_pret= pd.DataFrame(columns=['upper', 'lower','pred', 'obs'])

    result_total_pret['pred'] = df['pred']

    result_total_pret['lower'] =df['lower']
    result_total_pret['upper'] =df['upper']

    result_total_pret['obs']=test[:,0]

    if type=="mae":
        r2_total_pret =mean_absolute_error(test[:,0],result_total_pret['pred'])
    elif type=="r2":
        r2_total_pret =r2_score(test[:,0],result_total_pret['pred'])
    elif type=="mse":
        r2_total_pret =mean_squared_error(test[:,0],result_total_pret['pred'])
    elif type=="rmse":
        r2_total_pret=np.sqrt(mean_squared_error(test[:,0],result_total_pret['pred']))


    return r2_total_pret


from scipy.stats import norm
def evaluate_soil_property(model,
                         test,
                        sample_size = 100):

     df = pd.DataFrame(columns=['lower','upper','pred'])

     with torch.no_grad():

        model.eval()
        scaler = StandardScaler()
        scaler.fit(test)
        predictions=[]
        test = scaler.transform(test)
        test = torch.tensor(test, dtype=torch.float32)
        for _ in range(sample_size):

            output = model(test)  # Replace 'input_data' with your test input
            predictions.append(output.numpy())

        predictions = torch.tensor(predictions)

        prediction_mean = (torch.mean(predictions, dim=0)).detach().numpy()
        prediction_std = torch.std(predictions, dim=0).detach().numpy()

        # Calculate lower and upper bounds for the prediction interval (e.g., 95% interval)
        lower_bound_ = prediction_mean - 1.645 * prediction_std
        upper_bound_ = prediction_mean + 1.645 * prediction_std


        for i in range(0,len(prediction_mean)):

            #get the first element since we ar predicting just one at a time
            sample_pred = prediction_mean[i][0]

            lower_bound = lower_bound_[i][0]
            upper_bound = upper_bound_[i][0]

            row = {'upper':upper_bound,'lower':abs(lower_bound),'pred':sample_pred}

            df.loc[i] = row
        return df

## 2. Define Model

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal

class BayesianCNN(nn.Module):
    def __init__(self, num_feature: int, dims=[256, 512, 128, 32], weight_std=0.001, bias_std=9.5):
        super(BayesianCNN, self).__init__()

        # Define the fully connected layers based on the specified dimensions
        self.fc_layers = nn.ModuleList()
        in_dim = num_feature
        for out_dim in dims:
            self.fc_layers.append(nn.Linear(in_dim, out_dim))
            in_dim = out_dim

        self.fc_out = nn.Linear(in_dim, 1)  # Output layer for ph

        # Update the dimensions of weight_mu and weight_rho to match the output dimension
        self.weight_dim = out_dim
        self.weight_mu = nn.Parameter(torch.Tensor(1, self.weight_dim))
        self.weight_rho = nn.Parameter(torch.Tensor(1, self.weight_dim))

        self.bias_mu = nn.Parameter(torch.Tensor(1))
        self.bias_rho = nn.Parameter(torch.Tensor(1))
        self.weight_std = weight_std
        self.bias_std = bias_std

        # Initialize parameters
        self.reset_parameters()

    def reset_parameters(self):
        # Initialize weight means close to zero and standard deviations to be small
        nn.init.normal_(self.weight_mu, mean=0.0002, std=self.weight_std)
        nn.init.normal_(self.weight_rho, mean=0.003, std=self.bias_std)

        # Initialize bias means close to zero and standard deviations to be small
        nn.init.normal_(self.bias_mu, mean=0.0003, std=self.weight_std)
        nn.init.normal_(self.bias_rho, mean=0.004, std=self.bias_std)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input

        # Pass through fully connected layers with specified dimensions
        for fc_layer in self.fc_layers:
            x = F.rrelu(fc_layer(x))

        # Re-parameterization trick for sampling weights
        weight_epsilon = Normal(0, 1).sample(self.weight_mu.size())
        weight_sigma = torch.log(1 + torch.exp(self.weight_rho))
        weight = self.weight_mu + weight_sigma * weight_epsilon

        bias_epsilon = Normal(0, 1).sample(self.bias_mu.size())
        bias_sigma = torch.log(1 + torch.exp(self.bias_rho))
        bias = self.bias_mu + bias_sigma * bias_epsilon

        # Enforce non-negativity on weights and biases
        weight = torch.clamp(weight, min=0)
        bias = torch.clamp(bias, min=0)

        # Final linear layer operation
        output = F.linear(x, weight, bias)
        return output



In [8]:
train = torch.utils.data.TensorDataset(X_train,y_train)
train_dataloader = DataLoader(train, batch_size=16)

val= torch.utils.data.TensorDataset(X_val,y_val)
val_dataloader = DataLoader(val, batch_size=16)

test= torch.utils.data.TensorDataset(X_test,y_test)
test_dataloader = DataLoader(test, batch_size=16)

Using cpu device


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

#Manually compute the L1 loss over all model parameters:
def l1_penalty(model):
    l1_loss = 0.0
    for param in model.parameters():
        l1_loss += torch.abs(param).sum()
    return l1_loss

def train_epoch(train,val,model,loss_fn,optimizer,batch_size,n_epochs):

    train_dataloader = DataLoader(train, batch_size=batch_size)
    model.train()
    for epoch in range(n_epochs):
        for inputs, targets in train_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()

        # Print the training loss for this epoch
        print(f"Epoch [{epoch+1}/{n_epochs}] Loss: {loss.item():.5f}")
        #evaluate accuracy at end of each epoch
        test_epoch(val,model, loss_fn ,optimizer,batch_size,n_epochs)

def test_epoch(val,model,loss_fn,optimizer,batch_size,n_epochs):
    val_dataloader = DataLoader(val, batch_size=batch_size)

    val_loss=0
    model.train()

    for inputs, targets in val_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            val_loss += loss.item()
            loss.backward()
            optimizer.step()

    # Print the training loss for this epoch
    avg_val_loss = val_loss / len(inputs)
    print(f"Epoch [{n_epochs+1}/{n_epochs}] Val Loss: {loss.item():.5f}")


    return avg_val_loss


def get_metrics(test,y_test,model):
    loss_fn = nn.MSELoss()

    model.eval()

    y_pred = model(test)
    mse = loss_fn(y_pred, y_test)
    mse = float(mse)
    rmse= np.sqrt(mse)

    test_result= evaluate_soil_property(model,test)
    test_result["obs"] = y_test
    pcip = calculate_pcip(test_result["obs"], test_result['lower'], test_result['upper'])
    r2score = R2Score()
    r2= r2score(y_pred, y_test).item()

    r2_pent =r2*100
    #Balancing Trade-offs between r2 and pcip:
    score_avg = (pcip +  r2_pent)/2
    print(f'MSE: {mse:.4f},R2: {r2:.4f},RMSE: {rmse:.4f}, PCIP: {pcip:.2f},score_avg:{score_avg:.4f}')
    return [mse,r2,rmse,pcip]

def calculate_pcip(y_true, lower_bounds, upper_bounds):
    num_samples = len(y_true)
    num_covering_intervals = np.sum((lower_bounds <= y_true) & (y_true <= upper_bounds))
    pcip = (num_covering_intervals / num_samples) * 100
    return pcip

Using cpu device


## 2. Hyperparameter tuning to find best Architecture and parameters

In [29]:
from ray.tune.search.optuna import OptunaSearch
import torch
import torch.optim as optim
import ray
from ray.air import session, Checkpoint
from ray.tune.schedulers import PopulationBasedTraining, HyperBandForBOHB
from ray.tune.schedulers import AsyncHyperBandScheduler
from enum import Enum
loss_fn = nn.MSELoss()


class Optimiser(str, Enum):
    Adams = "Adams"
    SGD ="SGD"
    Adadelta = "Adadelta"
    Adagrad = "Adagrad"
    Adamax = "Adamax"
    Nadam = "Nadam"
    Ftrl = "Ftrl"
    RMSprop="RMSprop"
    LBFGS ="LBFGS"
    #LBFGS ="LBFGS"
    def __str__(self):
        return self.value

 # set your desired L1 regularization strength
def objective(config):  # ①

    #dataset
    train = torch.utils.data.TensorDataset(X_train,y_train)

    val= torch.utils.data.TensorDataset(X_val,y_val)


    output_dims=[2, 8, 16, 32, 64, 128, 256, 512,1024]

    #dim = random.sample(output_dims, config["n_layers"])

    criterion=nn.MSELoss()



    dims = config["dims"]  # You can change this to any number you want



    model = BayesianCNN(num_feature=X_train.shape[1],dims=dims,
                           weight_std=config["weight_std"],
                          bias_std=config["bias_std"]).to("cpu")   # Create a PyTorch conv net

    if config["optimiser"] == Optimiser.RMSprop:
        optimizer = torch.optim.RMSprop(model.parameters(), momentum=config["opt_momentum"], lr=config["lr"])
    elif config["optimiser"] == Optimiser.LBFGS:
        optimizer = torch.optim.LBFGS(model.parameters(),  lr=config["lr"])
    elif config["optimiser"] == Optimiser.Adams:
        optimizer = torch.optim.Adam(model.parameters(),  lr=config["lr"])
    elif config["optimiser"] == Optimiser.SGD:
        optimizer = torch.optim.SGD(model.parameters(),  lr=config["lr"],momentum=config["opt_momentum"])
    elif config["optimiser"] == Optimiser.Nadam:
        optimizer = torch.optim.NAdam(model.parameters(),  lr=config["lr"],momentum_decay=config["opt_momentum"])

    checkpoint = session.get_checkpoint()

    if checkpoint:
        checkpoint_state = checkpoint.to_dict()
        with checkpoint.as_directory() as dir_path:
            print("test")
            model_state, optimizer_state = torch.load(os.path.join(dir_path, "checkpoint.pt"))

            # Load optimizer state (needed since we're using momentum),
            # then set the `lr` and `momentum` according to the config.
            optimizer.load_state_dict(optimizer_state)
            model.load_state_dict(model_state)
        for param_group in optimizer.param_groups:
            if "lr" in config:
                param_group["lr"] = config["lr"]
            if "opt_momentum" in config:
                param_group["opt_momentum"] = config["opt_momentum"]

    while True:

        train_epoch(train,val,model, criterion ,optimizer,config["batch_size"],config["epochs"])  # Train the model

        os.makedirs("model", exist_ok=True)
        torch.save(
            (model.state_dict(), optimizer.state_dict()), "model/checkpoint.pt")

        mse,r2,rmse,pcip = get_metrics(X_test,y_test,model)  # Compute test accuracy
        #optimise with pcip and r2
        r2_pent =r2*100
        score_avg = (pcip+r2_pent)/2
        checkpoint = Checkpoint.from_directory("model")

        session.report({"done": pcip > 99 and rmse < 0.25,"mse": mse, "r2": r2,"rmse": rmse, "pcip": pcip,"score_avg":score_avg},checkpoint=checkpoint)  # Report to Tune

##### RUN  ##############
import ray
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.schedulers.pb2 import PB2
from ray.tune import tune
import random

ray.shutdown()
# Your original list of dimensions
original_dims = [512, 256,  128, 64, 32,  16,  8, 512,256,128,64,28]
# List of optimizer names
optimizer_names = ["Adams", "SGD","Nadam"]
optimizer_names = ["Adams", "Nadam"]
ray.init(num_cpus=12, num_gpus=0,_temp_dir="/ray") # assign the total # of cpus and gpus, make sure you have ray.init in the beginning and ray.shutdown at the end
sched = AsyncHyperBandScheduler(  time_attr="training_iteration",
    reduction_factor=2,
    metric='rmse',
    mode='min')  # set a scheduler

perturbation_interval = 10
#use population based training
scheduler =PopulationBasedTraining(
       time_attr="training_iteration",
    perturbation_interval=perturbation_interval,
    metric="rmse",
    mode="min",
    quantile_fraction=0.25,  # copy bottom % with top %
    #hyperparam_mutation - for pbt
    hyperparam_mutations={
        # distribution for resampling
        "lr": [0.0001, 0.1],
        "momentum": [0.009,0.01],
        "opt_momentum": [0.009,0.01],
    }
)
algo =OptunaSearch(metric=["rmse","pcip"], mode=["min","max"])# HyperOptSearch() # if you want to use the Bayesian optimization, import BayesOptSearch instead
algo = ConcurrencyLimiter(algo, max_concurrent=12)
scheduler = HyperBandForBOHB(
    time_attr="training_iteration",
    max_t=30,
    #metric="rmse",
    #mode='min',
    reduction_factor=0.25,
    stop_last_trials=False,
)
scheduler = PB2(
    time_attr="training_iteration",
    perturbation_interval=perturbation_interval,
    metric="score_avg",
    mode="max",
    quantile_fraction=0.4,  # copy bottom % with top %
    #hyperparam_mutation - for pbt
    hyperparam_bounds={
        # distribution for resampling
        "lr": [0.00001, 0.1],
        "momentum": [0.009,0.01],
        "opt_momentum": [0.009,0.01],
    },
)
analysis = tune.run(
        objective,   # the core training/testing of your model
        #storage_path=os.getcwd(), # for saving the log files
        name="pbt_ph", # name for the result directory
        #resume="REMOTE",
        #metric="rmse",
        resume="PROMPT",  #uncomment if you want to resume this experiment using the save check point
        #mode='min',
        #search_alg=algo,
        scheduler=scheduler,
        stop={
                "training_iteration": 20,
                "done": True,
                "rmse": 0.05,
        },
        resources_per_trial={
                "cpu": 1,
                "gpu": 0
         },
        num_samples=30, # 50 trials
        progress_reporter=ray.tune.JupyterNotebookReporter(metric="score_avg",overwrite=True,max_report_frequency=7),
        config={

               "lr": ray.tune.loguniform(0.00001, 0.1),
                "l1_coef": ray.tune.loguniform(0.00001, 0.3),
                  #"pred_interval_constant":ray.tune.loguniform(0.28, 12.05),
                 "momentum": ray.tune.loguniform(0.009,0.01),
                 "weight_std": ray.tune.loguniform(0.0001,10.02),
                 "bias_std": ray.tune.loguniform(0.01, 17.7),
                "dims": ray.tune.sample_from(lambda spec: random.choices(original_dims,k=random.randint(2, 7))),
                 "batch_size": ray.tune.randint(2,128),
                 "epochs":ray.tune.randint(2,1000),
                "optimiser": ray.tune.sample_from(lambda spec: random.choice(optimizer_names)),
                 "checkpoint_interval": perturbation_interval
        })


2023-09-30 16:21:21,243	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-09-30 16:21:21,575	WARNING callback.py:144 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2023-09-30 16:21:25,796	ERROR trial_runner.py:267 -- Runner restore failed.
Traceback (most recent call last):
  File "C:\Users\omondiagbep\Anaconda3\envs\pytorch\lib\site-packages\ray\tune\execution\trial_runner.py", line 258, in __init__
    self.resume(
  File "C:\Users\omondiagbep\Anaconda3\envs\pytorch\lib\site-packages\ray\tune\execution\trial_runner.py", line 506, in resume
    trials = self.restore_from_dir()
  File "C:\Users\omondiagbep\Anaconda3\envs\pytorch\lib\s

Resume from local directory? (C:\Users\omondiagbep\ray_results\pbt_ph) [y/N]:(objective pid=38580) Epoch [1/175] Loss: 13.20879
(objective pid=38580) Epoch [176/175] Val Loss: 0.97571
(objective pid=29704) Epoch [1/533] Loss: 29.76322 [repeated 10x across cluster]
(objective pid=29704) Epoch [534/533] Val Loss: 26.83703 [repeated 10x across cluster]
(objective pid=29704) Epoch [4/533] Loss: 1.40976 [repeated 25x across cluster]
(objective pid=22480) Epoch [800/799] Val Loss: 1.74622 [repeated 23x across cluster]
(objective pid=37044) Epoch [5/484] Loss: 1.38364 [repeated 25x across cluster]
(objective pid=38580) Epoch [176/175] Val Loss: 4.97641 [repeated 25x across cluster]
(objective pid=38580) Epoch [9/175] Loss: 6.39658 [repeated 29x across cluster]
(objective pid=29704) Epoch [534/533] Val Loss: 5.72545 [repeated 28x across cluster]
(objective pid=4568) Epoch [13/914] Loss: 0.18073 [repeated 25x across cluster]
(objective pid=27472) Epoch [23/22] Val Loss: 5.79101 [repeated 25x ac

Trial name,mse,pcip,r2,rmse,score_avg,should_checkpoint
objective_6d8f0_00010,0.848394,99.5565,-0.244564,0.921083,37.55,True


In [40]:
best_trials = analysis.get_best_trial("score_avg", "max", "all")
print(f"Best trial config: {best_trials.config}")
print(f"Best trial final validation mse: {best_trials.last_result['mse']}")
print(f"Best trial final validation r2: {best_trials.last_result['r2']}")
print(f"Best trial final validation rmse: {best_trials.last_result['rmse']}")
print(f"Best trial final validation pcip: {best_trials.last_result['pcip']}")
#print(f"Best trial final validation score_avg: {best_trials.last_result['optimal_dim']}")
#print(f"Best trial final validation l1_coef: {best_trial.last_result['l1_coef']}")
best_checkpoints = best_trials.checkpoint.to_air_checkpoint()


Best trial config: {'lr': 4.708890655042492e-05, 'l1_coef': 0.1555100205034113, 'momentum': 0.009621543799742525, 'weight_std': 1.2993019125547687, 'bias_std': 0.2769466697037927, 'dims': [512, 64, 128], 'batch_size': 58, 'epochs': 799, 'optimiser': 'Adams', 'checkpoint_interval': 10, 'opt_momentum': 0.009750988247210268}
Best trial final validation mse: 0.09911589324474335
Best trial final validation r2: 0.8546004891395569
Best trial final validation rmse: 0.3148267670398172
Best trial final validation pcip: 98.22616407982262


In [42]:
gpus_per_trial = 2
best_trained_model=BayesianCNN(num_feature=X_train.shape[1], dims=best_trials.config["dims"]
                           ,weight_std=best_trials.config["weight_std"],
                          bias_std=best_trials.config["bias_std"])

device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if gpus_per_trial > 1:
        best_trained_model = nn.DataParallel(best_trained_model)
best_trained_model.to(device)
with best_checkpoints.as_directory() as dir_path:
    print(dir_path)
    model_state, optimizer_state = torch.load(os.path.join(dir_path, "checkpoint.pt"))
    best_trained_model.load_state_dict(model_state)
best_checkpoint_data = best_checkpoints.to_dict()


C:\Users\omondiagbep\ray_results\pbt_ph\objective_6d8f0_00006_6_batch_size=58,bias_std=0.2769,epochs=799,l1_coef=0.1555,lr=0.0000,momentum=0.0096,weight_std=1.2993_2023-09-30_16-21-32\checkpoint_000000


In [42]:


def train_func(model,optimizer_=None,n_epochs = 1000,batch_size = 16,lr=0.0001):
    loss_fn = nn.MSELoss()  # mean square error

    if optimizer_ is None:
        optimizer = torch.optim.Adam(model.parameters(),  lr=lr)
    if optimizer_ == Optimiser.RMSprop:
        optimizer = torch.optim.RMSprop(model.parameters(), momentum=best_trials.config["opt_momentum"], lr=best_trials.config["lr"])
    elif optimizer_ == Optimiser.LBFGS:
        optimizer = torch.optim.LBFGS(model.parameters(),  lr=best_trials.config["lr"])
    elif optimizer_ == Optimiser.Adams:
        optimizer = torch.optim.Adam(model.parameters(),  lr=best_trials.config["lr"])
    elif optimizer_ == Optimiser.SGD:
        optimizer = torch.optim.SGD(model.parameters(),  lr=best_trials.config["lr"],momentum=best_trials.config["opt_momentum"])
    elif optimizer_ == Optimiser.Nadam:
        optimizer = torch.optim.NAdam(model.parameters(),  lr=best_trials.config["lr"],momentum_decay=best_trials.config["opt_momentum"])


    # Hold the best model
    best_mse = np.inf # init to infinity

    history = []
    epochs_no_improve = 0

    early_stop=False
    n_epochs_stop = 40
    min_val_loss=float('inf')
    best_r2 = 0.0000
    best_pcip = 0.0000
    criterion=nn.MSELoss()

    train = torch.utils.data.TensorDataset(X_train,y_train)
    val = torch.utils.data.TensorDataset(X_val,y_val)
    train_dataloader = DataLoader(train, batch_size=batch_size)
    val_dataloader = DataLoader(val, batch_size=batch_size)
    for epoch in range(n_epochs):
        model.train()
        train_loss =0
        val_loss=0

        for inputs, targets in train_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        # Print the training loss for this epoch
        print(f"Epoch [{epoch+1}/{n_epochs}] Loss: {loss.item():.5f}")

        for inputs, targets in val_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            loss.backward()
            optimizer.step()

        # Print the training loss for this epoch
        avg_val_loss = val_loss / len(inputs)
        print(f"Epoch [{epoch+1}/{n_epochs}] Val Loss: {loss.item():.5f}")

        #at the start
        if avg_val_loss < min_val_loss:
            epochs_no_improve = 0
            min_val_loss = avg_val_loss
            print("restarting counter")
            print(epochs_no_improve)
        else:
            epochs_no_improve += 1
            print("increasing counter")
            print(epochs_no_improve)

        if epochs_no_improve == n_epochs_stop:
            print('Early stopping!')
            early_stop = True
            break
        # evaluate accuracy at end of each epoch
        model.eval()
        mse,r2,rmse,pcip = get_metrics(X_test,y_test,model)
        print(f'r2: {r2:.4f}')
        print(f'pcip: {pcip:.4f}')
        y_pred = model(X_test)

        mse = loss_fn(y_pred, y_test)
        mse = float(mse)
        rmse= np.sqrt(mse)
        print(f'RMSE: {rmse:.4f}')
        history.append(rmse)
        if rmse < best_mse:
            best_mse = rmse

        if r2 > best_r2 :
            best_r2 = r2
            print("save rmse state")
            modelrmse_state_dict = model.state_dict()
        if pcip > best_pcip :
            best_pcip = pcip
            print("save pcip state")
            modelpcip_state_dict = model.state_dict()

        if early_stop:
            print('Training stopped early.')


        print(f'Best RMSE so far: {best_mse:.4f}')
        print(f'Best r2 so far: {best_r2:.4f}')

        print(f'Best pcip so far: {best_pcip:.4f}')


    modelrmse_state_dict.update(modelpcip_state_dict)

    # Load the updated state_dict back into model
    model.load_state_dict(modelrmse_state_dict)
    print("RMSE: %.2f" % best_mse)
    plt.plot(history)
    plt.show()
    return model,history,best_mse

## Best architecture found

In [47]:

#fine tune if needed

model = BayesianCNN(num_feature=X_train.shape[1],
                       weight_std=best_trials.config["weight_std"],
                      bias_std=best_trials.config["bias_std"]).to("cpu")

In [48]:
model,history,best_mse =train_func(model,n_epochs=best_trials.config["n_epochs"],batch_size=best_trials.config["batch_size"], lr=best_trials.config["lr"])

Epoch [1/10] Loss: 12.59153
Epoch [1/10] Val Loss: 1338.84766
restarting counter
0
MSE: 230.3940,R2: -336.9799,RMSE: 15.1787, PCIP: 0.00,score_avg:-16848.9944
r2: -336.9799
pcip: 0.0000
RMSE: 25.7670
Best RMSE so far: 25.7670
Best r2 so far: 0.0000
Best pcip so far: 0.0000
Epoch [2/10] Loss: 21.96436
Epoch [2/10] Val Loss: 564.36646
restarting counter
0
MSE: 42.5900,R2: -61.4780,RMSE: 6.5261, PCIP: 0.00,score_avg:-3073.9002
r2: -61.4780
pcip: 0.0000
RMSE: 9.0873
Best RMSE so far: 9.0873
Best r2 so far: 0.0000
Best pcip so far: 0.0000
Epoch [3/10] Loss: 170.60124
Epoch [3/10] Val Loss: 90.66891
increasing counter
1


KeyboardInterrupt: 

In [61]:
#save model
best_model_name = f'mir_ph_model-1.0.1.pt'
torch.save(best_trained_model.state_dict(),os.path.join("../../model-store/",best_model_name))



In [56]:
#load model if testing
best_model_name = f'mir_ph_model-1.0.0.pt'
best_trained_model=torch.load(os.path.join("../../model-store/",best_model_name))

In [63]:
best_trials.config["dims"]

[512, 64, 128]

In [57]:
## Testing
best_trained_model.eval()

BayesianCNN(
  (fc_layers): ModuleList(
    (0): Linear(in_features=340, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=128, bias=True)
  )
  (fc_out): Linear(in_features=128, out_features=1, bias=True)
)

In [58]:
with torch.no_grad():
    # Test out inference with 5 samples
    for i in range(5):
        X_sample = X_test_raw[i: i+1]
        X_sample = scaler.transform(X_sample)
        X_sample = torch.tensor(X_sample, dtype=torch.float32)
        y_pred= best_trained_model(X_sample)

        print(f"{y_pred.numpy()} (expected {y_test[i].numpy()})")


[[6.194552]] (expected [5.9])
[[7.5351286]] (expected [7.3])
[[6.777897]] (expected [6.3])
[[6.4349313]] (expected [6.])
[[5.765042]] (expected [6.2])


In [43]:
test_result= evaluate_soil_property(best_trained_model,X_test_raw,sample_size=1000)
test_result["obs"] = y_test
test_result['type'] ='Validation'
test_result['variable'] ='pH'
test_result

,lower,upper,pred,obs,type,variable
0,5.328810,6.683116,6.005963,5.9,Validation,pH
1,6.048503,7.483996,6.766250,7.3,Validation,pH
2,5.565150,6.948915,6.257032,6.3,Validation,pH
3,5.528523,6.897013,6.212768,6.0,Validation,pH
4,5.472255,6.862381,6.167318,6.2,Validation,pH
...,...,...,...,...,...,...
897,4.810188,6.118444,5.464316,5.7,Validation,pH
898,6.000397,7.418135,6.709266,6.8,Validation,pH
899,4.938396,6.278067,5.608232,5.5,Validation,pH
900,4.297469,5.567361,4.932415,5.3,Validation,pH


In [44]:
train_result= evaluate_soil_property(best_trained_model,X_train_raw,sample_size=1000)
train_result["obs"] = y_train
train_result['type'] ='Calibration'
train_result['variable'] ='pH'
train_result

,lower,upper,pred,obs,type,variable
0,7.191580,8.832384,8.011982,8.1,Calibration,pH
1,5.124605,6.510371,5.817488,5.6,Calibration,pH
2,5.474174,6.891250,6.182712,6.1,Calibration,pH
3,5.165444,6.601531,5.883488,5.8,Calibration,pH
4,4.394381,5.759455,5.076918,5.1,Calibration,pH
...,...,...,...,...,...,...
2161,5.882193,7.457187,6.669690,6.0,Calibration,pH
2162,6.128698,7.656775,6.892736,6.6,Calibration,pH
2163,6.505368,8.075612,7.290490,7.2,Calibration,pH
2164,5.620674,7.077077,6.348876,6.3,Calibration,pH


In [54]:
final_df=pd.merge(train_result,test_result,how="outer")

In [55]:
final_df.to_csv("../../CNN/cnn_ph.csv")
final_df

,lower,upper,pred,obs,type,variable
0,7.191580,8.832384,8.011982,8.1,Calibration,pH
1,5.124605,6.510371,5.817488,5.6,Calibration,pH
2,5.474174,6.891250,6.182712,6.1,Calibration,pH
3,5.165444,6.601531,5.883488,5.8,Calibration,pH
4,4.394381,5.759455,5.076918,5.1,Calibration,pH
...,...,...,...,...,...,...
3063,4.810188,6.118444,5.464316,5.7,Validation,pH
3064,6.000397,7.418135,6.709266,6.8,Validation,pH
3065,4.938396,6.278067,5.608232,5.5,Validation,pH
3066,4.297469,5.567361,4.932415,5.3,Validation,pH


In [45]:
import numpy as np

def calculate_pcip(y_true, lower_bounds, upper_bounds):
    num_samples = len(y_true)
    num_covering_intervals = np.sum((lower_bounds <= y_true) & (y_true <= upper_bounds))
    pcip = (num_covering_intervals / num_samples) * 100
    return pcip

pcip = calculate_pcip(test_result['obs'], test_result['lower'], test_result['upper'])
print(f"Prediction Interval Coverage Probability (PCIP): {pcip:.2f}%")

Prediction Interval Coverage Probability (PCIP): 97.23%


In [46]:
#mean predition interval width
def calculate_mpiw(lower_bounds, upper_bounds):
    num_samples = len(lower_bounds)
    num_covering_intervals = np.sum(upper_bounds -lower_bounds)
    mpiw = (num_covering_intervals / num_samples)
    return mpiw

mpiw = calculate_mpiw(test_result['lower'], test_result['upper'])
print(f"Mean Prediction Interval Width (mpiw): {mpiw:.2f}")

Mean Prediction Interval Width (mpiw): 1.34


In [48]:
evaluate(best_trained_model,X_train_raw,y_train,type="r2")

0.9592606443231444

In [47]:
evaluate(best_trained_model,X_test_raw,y_test,type="r2")

0.8713408643995437

In [49]:
evaluate(best_trained_model,X_test_raw,y_test,type="rmse")

0.2920202

In [50]:
evaluate(best_trained_model,X_train_raw,y_train,type="rmse")


0.16807888